In [1]:
import rvnewop as rv
from glob import glob
import matplotlib.pyplot as plt
from itertools import accumulate

In [3]:
# remove st.hst, edn.hst, minver.hst, cubic.hst, wikisort.hst, nbody.hst
floating_point_hsts = list(map(lambda x: "./embench_hst/"+x, ["st.hst", "edn.hst", "minver.hst", "cubic.hst", "wikisort.hst", "nbody.hst"]))
for FILENAME in [x for x in glob("./embench_hst/*.hst") if x not in floating_point_hsts]:
    #print("Processing {}...".format(FILENAME))

    prog = rv.Histogram.parse(FILENAME, isa="32ICM")
    prog.findBasicBlocks()

    prog.addLivenessValuesToGraph()

    # total cycles = sum of all the frequencies
    total_cycles = sum(
        [inst.freq for bb in prog.basicBlocks for inst in bb.bbInstructions()]
    )

    new_instructions = []

    for i, b in enumerate((prog.getSubBlocks())):
        graph = b.constructDAG()

        # temp will store every permutation of candidate subgraphs
        temp = []
        for n in rv.analysis.findCandidateSubgraphs(prog, graph):
            subtree = rv.analysis.createSubtreeFromNode(graph, n)
            root = n

            temp.append(rv.Subgraph(subtree, root))

        # add candidate subgraph with highest score to new instructions list
        current_nodes = set()
        temp = sorted(temp, key=lambda sg: sg.score, reverse=True)
        while temp:
            if any([(n in current_nodes) for n in temp[0].graph.nodes]):
                temp.pop(0)
            else:
                current_nodes.update(temp[0].graph.nodes)
                new_instructions.append(temp.pop(0))

    new_instructions = sorted(new_instructions, key=lambda sg: sg.score, reverse=True)

    inst_dict = {}
    for subgraph in new_instructions:
        key = rv.analysis.graphToParenString(subgraph.graph)

        if key in inst_dict:
            inst_dict[key].score += subgraph.score
        else:
            inst_dict[key] = subgraph

    new_instructions = [inst_dict[key] for key in inst_dict]
    new_instructions = sorted(new_instructions, key=lambda sg: sg.score, reverse=True)

    max_depth = 0
    max_mult_depth = 0
    depth_hst = {x:0 for x in range(8)}
    for sg in new_instructions:
        
        depth_hst[min(sg.depth, 7)] += 1
        
        if sg.containsMultiplyInstruction():
            max_mult_depth = max(max_mult_depth, sg.depth)
        else:
            max_depth = max(max_depth, sg.depth)

    saved_cycles = sum([sg.score for sg in new_instructions])
    percent_cycles = float(saved_cycles) / total_cycles * 100
    #plt.clf()
    #plt.plot([x for x in range(len(new_instructions))],list(map(lambda x:x/total_cycles, accumulate([sg.score for sg in new_instructions]))))
    #plt.show()
    #print(
    #    """Saved {} cycles out of {} ({:.4}%) with {} new instructions""".format(
    #        saved_cycles, total_cycles, percent_cycles, len(new_instructions)
    #    )
    #)
    #print("max depth: {}\nmax mult depth: {}".format(max_depth, max_mult_depth))
    
    print("{},{:.4},{:.4},{:.4},{:.4},{:.4},{:.4},{:.4}".format(FILENAME[len("./embench_hst/"):-4], *[depth_hst[key]/len(new_instructions) for key in range(8)]))


aha-mont64,0.0,0.0,0.4,0.3333,0.1111,0.1333,0.02222
ud,0.0,0.0,0.3478,0.3261,0.1522,0.1522,0.02174
slre,0.0,0.0,0.3265,0.3673,0.1633,0.1224,0.02041
qrduino,0.0,0.0,0.2381,0.3333,0.2262,0.1429,0.03571
matmult-int,0.0,0.0,0.3469,0.3061,0.1633,0.1429,0.02041
picojpeg,0.0,0.0,0.2299,0.4023,0.09195,0.1494,0.05747
sglib-combined,0.0,0.0,0.3509,0.3509,0.1404,0.1404,0.0
nsichneu,0.0,0.0,0.381,0.3333,0.119,0.1429,0.02381
coremark,0.0,0.0,0.3158,0.3289,0.2237,0.1053,0.01316
nettle-sha256,0.0,0.0,0.375,0.3542,0.125,0.125,0.02083
huffbench,0.0,0.0,0.3443,0.4098,0.1311,0.1148,0.0
crc32,0.0,0.0,0.3333,0.3333,0.1333,0.1556,0.04444
nettle-aes,0.0,0.0,0.3654,0.3462,0.1346,0.1346,0.01923
statemate,0.0,0.0,0.3488,0.3488,0.1395,0.1395,0.02326
